In [1]:
script = False

import pandas as pd
import numpy as np
import random
import os

from astropy.io import fits
from astropy.table import Table

import matplotlib.pyplot as plt
from astropy.visualization import make_lupton_rgb
plt.style.use('dark_background')

if(script):
    from tqdm import tqdm
else: 
    from tqdm.notebook import tqdm

In [2]:
def make_one_plot(objects, title, data, Q, stretch, minimum):
    plt.figure(figsize=(4,4))
    rgb = make_lupton_rgb(objects[2], objects[1], objects[0], Q=Q, stretch=stretch, minimum=minimum)
    rgb = np.flipud(rgb)
    
    plt.imshow(rgb, aspect='equal')
    plt.xticks([], [])
    plt.yticks([], []) 
    plt.savefig(png_path + title, bbox_inches='tight')
    #plt.show()
    plt.close()
    
def make_one_plot_brit(objects, title, data):
    plt.figure(figsize=(4,4))
    
    imager = 0.0005*objects[2] 
    imageg = 3*objects[1] 
    imageb = 15*objects[0] 
    rgb = np.array([imager, imageg, imageb])
    #Making sure values are lower or equal to 255
    rgb = np.clip(rgb, 0, 255).astype(np.uint8)
    rgb = rgb.transpose(1, 2, 0)
    #Increasing contrast
    min_img, max_img = np.min(rgb), np.max(rgb)     
    LUT=np.zeros(256,dtype=np.uint8)
    LUT[min_img:max_img+1]=np.linspace(start=0,stop=255,num=(max_img-min_img)+1,endpoint=True,dtype=np.uint8)
    rgb = LUT[rgb]
    rgb = np.flipud(rgb)
                
    plt.imshow(rgb, aspect='equal')
    plt.xticks([], [])
    plt.yticks([], []) 
    
    plt.savefig(png_path + title, bbox_inches='tight')
    #plt.show()
    plt.close()

In [3]:
N = 1000
filepath = 'fit_files/' + 'single.fits'
hdu_list = fits.open(filepath)
images = hdu_list[1].data#.data[0:N]
data = pd.DataFrame(hdu_list[2].data)#.iloc[0:N]
hdu_list.close()

print(len(images), len(data))

20026 20026


In [4]:
data['#subject_id'] = range(0, len(data))
data['#NEW_ID'] = range(0, len(data))
data['#image_name_1'] = data['#NEW_ID'].apply(lambda x: f'pngs/sing_{x}_1.png')
data['#image_name_2'] = data['#NEW_ID'].apply(lambda x: f'pngs/sing_{x}_2.png')
data['#image_name_3'] = data['#NEW_ID'].apply(lambda x: f'pngs/sing_{x}_3.png')
data['#image_name_4'] = data['#NEW_ID'].apply(lambda x: f'pngs/sing_{x}_4.png')
data['!Subject Type'] = ['Test Subject']*len(data)
data['#Type'] = ['SUB']*len(data)
data['#SubType'] = ['SUB']*len(data)
data['#Provenance'] = ['DES-VT-JG']*len(data)
data['Colours'] = ['gri']*len(data)
data['Size-in-arcsec'] = ['12']*len(data)
data['#CLASS'] = ['single_cand']*len(data)
data['#REPORTED'] = [False]*len(data)
data['#REF'] = [None]*len(data)
data['!Previously published lens candidate?'] = ['No']*len(data)

In [5]:
#Adding columns for when they have been reported previously and the reference
filepath = '/Users/jimenagonzalez/research/DSPL/Searching-double-lenses/vit_cnn_multiclass/results/data/'
known_data = pd.read_csv(filepath + 'SLED_lens_database.csv')
size = 0.0032
counter = 0

for i in range(len(data)):
    ra, dec = data['RA'].iloc[i], data['DEC'].iloc[i]
    data_tmp = known_data[known_data['ra'] > ra - size][known_data['ra'] < ra + size][known_data['dec'] > dec - size][known_data['dec'] < dec + size]
    if(len(data_tmp) > 0):
        counter += 1
        data['#REPORTED'].iloc[i] = True
        data['#REF'].iloc[i] = data_tmp['bibcode'].iloc[0]
        data['!Previously published lens candidate?'].iloc[i] = 'Yes, it is listed in a published paper, but may be a high or low probability candidate'
        
print('Already reported: ', counter)

/Users/jimenagonzalez/Software/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == "__main__":
/Users/jimenagonzalez/Software/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Already reported:  1351


In [6]:
col_drop = ['COADD_OBJECT_ID', 'TILENAME', 'PROB', 'HPIX_16384', 'HPIX_4096', 'DNF_ZMEAN_SOF', 'RA', 'DEC', 
        'FLUX_RADIUS_G', 'FLUX_RADIUS_R', 'FLUX_RADIUS_I', 'FLUX_RADIUS_Z', 'KRON_RADIUS', 'GAP_FLUX_G',
       'MOF_BDF_FLUX_G', 'MOF_PSF_FLUX_G', 'SOF_BDF_FLUX_G', 'SOF_PSF_FLUX_G', 'MAG_AUTO_G', 'MAG_APER_4_G', 
        'MAG_APER_8_G', 'SOF_BDF_G_1','SOF_BDF_G_2', 'IMAFLAGS_ISO_G', 'IMAFLAGS_ISO_R', 'IMAFLAGS_ISO_I',
       'IMAFLAGS_ISO_Z', 'EXT_COADD']
new_data = data.drop(col_drop, axis=1)

path = 'to_upload/single/'
data.to_csv(path + 'complete_manifest.csv', index=False)
new_data.to_csv(path + 'manifest.csv', index=False)

print(new_data.columns)
print(data.columns)

Index(['#subject_id', '#NEW_ID', '#image_name_1', '#image_name_2',
       '#image_name_3', '#image_name_4', '!Subject Type', '#Type', '#SubType',
       '#Provenance', 'Colours', 'Size-in-arcsec', '#CLASS', '#REPORTED',
       '#REF', '!Previously published lens candidate?'],
      dtype='object')
Index(['COADD_OBJECT_ID', 'TILENAME', 'PROB', 'HPIX_16384', 'HPIX_4096',
       'DNF_ZMEAN_SOF', 'RA', 'DEC', 'FLUX_RADIUS_G', 'FLUX_RADIUS_R',
       'FLUX_RADIUS_I', 'FLUX_RADIUS_Z', 'KRON_RADIUS', 'GAP_FLUX_G',
       'MOF_BDF_FLUX_G', 'MOF_PSF_FLUX_G', 'SOF_BDF_FLUX_G', 'SOF_PSF_FLUX_G',
       'MAG_AUTO_G', 'MAG_APER_4_G', 'MAG_APER_8_G', 'SOF_BDF_G_1',
       'SOF_BDF_G_2', 'IMAFLAGS_ISO_G', 'IMAFLAGS_ISO_R', 'IMAFLAGS_ISO_I',
       'IMAFLAGS_ISO_Z', 'EXT_COADD', '#subject_id', '#NEW_ID',
       '#image_name_1', '#image_name_2', '#image_name_3', '#image_name_4',
       '!Subject Type', '#Type', '#SubType', '#Provenance', 'Colours',
       'Size-in-arcsec', '#CLASS', '#REPORTED', '#REF'

In [7]:
filepath = '/Users/jimenagonzalez/research/DSPL/SpaceWarps_Inspection/initial/for_test/results/'
to_compare = pd.read_csv(filepath + 'Manifest_DES-VT-BetaTest.csv')

df1, df2 = new_data, to_compare
columns_in_df1_only = set(df1.columns) - set(df2.columns)
columns_in_df2_only = set(df2.columns) - set(df1.columns)

print("Columns unique to df1:", columns_in_df1_only)
print("Columns unique to df2:", columns_in_df2_only)

Columns unique to df1: {'#REF', '#image_name_4', '#subject_id', '#REPORTED'}
Columns unique to df2: {'subject_id', '#Lens-candidate', '#KNOWN_LENS_REF', '#PROB'}


In [9]:
png_path = 'to_upload/single/'

for i in tqdm(range(len(data))):
    if(i == 5): break
    path_exists = png_path + data['#image_name_4'].iloc[i]
    if(os.path.exists(path_exists)):
        continue
    make_one_plot(images[i], data['#image_name_1'].iloc[i], data.iloc[i], 9, 60, (0,0,0)) # new first filter
    make_one_plot(images[i], data['#image_name_2'].iloc[i], data.iloc[i], 8, 40, (14,2,0)) # new second filter
    make_one_plot(images[i], data['#image_name_3'].iloc[i], data.iloc[i], 10, 37, (12,6,0)) # new third filter
    make_one_plot_brit(images[i], data['#image_name_4'].iloc[i], data.iloc[i])

  0%|          | 0/20026 [00:00<?, ?it/s]